In [43]:
import os
import numpy as np
import librosa
import pandas as pd
import soundfile as sf

In [2]:
current_wrd = os.getcwd()
dataset_path = os.path.join(current_wrd, 'recordings')
dataset_files = os.listdir(dataset_path)
dataset_files[:5]
# {digitLabel}_{speakerName}_{index}.wav


['0_george_0.wav',
 '0_george_1.wav',
 '0_george_10.wav',
 '0_george_11.wav',
 '0_george_12.wav']

In [25]:
df = pd.DataFrame({"file": dataset_files})
df["length"] = df["file"].apply(lambda x: librosa.core.load(os.path.join(dataset_path, x), sr=8000)[0].shape[0])
df["number"] = df["file"].apply(lambda x: x.split("_")[0])
df["speaker"] = df["file"].apply(lambda x: x.split("_")[1])
df["index"] = df["file"].apply(lambda x: x.split("_")[2].split(".")[0])


3135    6
3599    6
3410    6
3355    5
2936    5
       ..
1433    1
2837    1
1403    1
1634    1
3507    1
Name: length, Length: 2056, dtype: int64

In [10]:
df

,file,S,y,sr,ref_value,S_db,number,speaker,index,target
0,0_george_0.wav,None,None,None,None,None,0,george,0,0
1,0_george_1.wav,None,None,None,None,None,0,george,1,0
2,0_george_10.wav,None,None,None,None,None,0,george,10,0
3,0_george_11.wav,None,None,None,None,None,0,george,11,0
4,0_george_12.wav,None,None,None,None,None,0,george,12,0
...,...,...,...,...,...,...,...,...,...,...
2995,9_yweweler_5.wav,None,None,None,None,None,9,yweweler,5,0
2996,9_yweweler_6.wav,None,None,None,None,None,9,yweweler,6,0
2997,9_yweweler_7.wav,None,None,None,None,None,9,yweweler,7,0
2998,9_yweweler_8.wav,None,None,None,None,None,9,yweweler,8,0


In [50]:
import librosa
import numpy as np



source_file = os.path.join(dataset_path, "0_george_0.wav")
target_file = os.path.join(dataset_path, "0_lucas_0.wav")

#loading the audio files
michael_audio, michael_sr = librosa.load(source_file)
george_audio, george_sr = librosa.load(target_file)

#extracting the mfcc features
michael_mfcc = librosa.feature.mfcc(y=michael_audio, sr=michael_sr)
george_mfcc = librosa.feature.mfcc(y=george_audio, sr=george_sr)

#calculating the mean and standard deviation of the mfcc features
michael_mean = np.mean(michael_mfcc, axis=1)[0]
michael_std = np.std(michael_mfcc, axis=1)[0]
george_mean = np.mean(george_mfcc, axis=1)[0]
george_std = np.std(george_mfcc, axis=1)[0]

#normalizing the mfcc features
michael_mfcc_norm = (michael_mfcc - michael_mean) / michael_std
george_mfcc_norm = (george_mfcc - george_mean) / george_std

#applying the normalization to the michael audio
michael_audio_norm = michael_audio * michael_mfcc_norm

#synthesizing the audio
george_audio_norm = librosa.core.synthesize(george_mfcc_norm, george_sr)

#saving the audio
output_file = 'output_file_0_lucas_0.wav'

sf.write(output_file, george_audio_norm, george_sr, subtype='PCM_24')
# librosa.output.write_wav('george_voice.wav', george_audio_norm, george_sr)

#printing the output
print('Voice conversion successful!')

ValueError: operands could not be broadcast together with shapes (6571,) (20,13) 

In [52]:
import librosa
import numpy as np
import soundfile as sf

# Load the source and target audio files
source_file = os.path.join(dataset_path, "0_george_0.wav")
target_file = os.path.join(dataset_path, "0_lucas_0.wav")
source, sr = librosa.load(source_file)
target, _ = librosa.load(target_file, sr=sr, duration=len(source)/sr)

# Compute the LPC coefficients of the source audio file
order = 12
lpc_coeffs = librosa.lpc(y=source, order=order)

# 1. Spectral Envelope Transfer
# Compute the spectral envelope of the target audio file
target_spectrum = librosa.stft(y=target)
target_magnitude, target_phase = librosa.magphase(target_spectrum)
target_spectrum_envelope = librosa.amplitude_to_db(target_magnitude)

# Apply the spectral envelope of the target audio file to the source audio file
source_spectrum = librosa.stft(y=source)
source_magnitude, source_phase = librosa.magphase(source_spectrum)
output_spectrum = librosa.db_to_amplitude(target_spectrum_envelope) * source_phase
output_audio = librosa.istft(output_spectrum, length=len(source))

# Write the output to a file
output_file = 'output_file_spectral_envelope_transfer.wav'
sf.write(output_file, output_audio, sr)

# 2. Pitch Scaling
# Compute the pitch of the source audio file
source_pitch, _ = librosa.core.piptrack(y=source, sr=sr)
source_pitch = source_pitch[:,0]

# Compute the pitch of the target audio file
target_pitch, _ = librosa.core.piptrack(y=target, sr=sr)
target_pitch = target_pitch[:,0]

# Resample the source audio file to match the target pitch
output_audio = librosa.effects.pitch_shift(y=source, sr=sr, n_steps=librosa.hz_to_semitone(target_pitch/source_pitch), bins_per_octave=12)

# Write the output to a file
output_file = 'output_file_pitch_scaling.wav'
sf.write(output_file, output_audio, sr)

# 3. Formant Shifting
# Compute the formants (resonant frequencies) of the source audio file
roots = np.roots(lpc_coeffs)
angles = np.angle(roots)
freqs = angles * (sr / (2 * np.pi))
sorted_freqs = np.sort(freqs)

# Compute the target formants (e.g., the formants of the target speaker)
target_formants = [800, 1200, 2800]  # Replace with the desired target formants

# Modify the formants of the source audio file to match the target formants
shifted_roots = np.exp(1j * 2 * np.pi * np.array(target_formants) / sr)
shifted_coeffs = np.poly(shifted_roots)
shifted_audio = librosa.lfilter(b=shifted_coeffs, a=[1], x=source)

# Write the output to a file
output_file = 'output_file_formant_shifting.wav'
sf.write(output_file, shifted_audio, sr)


AttributeError: No librosa attribute hz_to_semitone